In [2]:
from ipyleaflet import (Map, GeoData, basemaps, WidgetControl, GeoJSON,
                        LayersControl, Icon, Marker,basemap_to_tiles, Choropleth,
                        MarkerCluster, Heatmap,SearchControl, 
                        FullScreenControl)

from ipywidgets import Text, HTML
from branca.colormap import linear
import geopandas as gpd
import json

In [3]:
# Countries
countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Conflict Dataset Points
africa_acled = gpd.read_file(
    "data/acled2019.shp",
    mask = countries[countries["continent"] == "Africa"]
)
africa_acled

In [4]:
africa_acled.head()

,data_id,iso,event_id_c,event_id_n,event_date,year,time_preci,event_type,sub_event_,actor1,...,latitude,longitude,geo_precis,source,source_sca,notes,fatalities,timestamp,iso3,geometry
0,6714129,706,SOM30213,30213,31 December 2019,2019,1,Battles,Armed clash,Al Shabaab,...,2.3963,45.0159,1,Undisclosed Source,Local partner-Other,"31 December 2019. In Warmahan, Al Shabaab mili...",0,1578512393,SOM,POINT (45.01590 2.39630)
1,6723349,12,ALG7951,7951,31 December 2019,2019,1,Protests,Peaceful protest,Protesters (Algeria),...,36.7525,3.0420,1,Liberte (Algeria); TSA Algerie; El Watan (Alge...,National,"On 31 December 2019, students held their 45th ...",0,1579554007,DZA,POINT (3.04200 36.75250)
2,6723350,12,ALG7953,7953,31 December 2019,2019,1,Protests,Peaceful protest,Protesters (Algeria),...,36.3650,6.6147,1,Liberte (Algeria); El Watan (Algeria),National,"On 31 December 2019, students held their 45th ...",0,1579554007,DZA,POINT (6.61470 36.36500)
3,6713368,404,KEN7182,7182,31 December 2019,2019,1,Violence against civilians,Attack,Police Forces of Kenya (2013-),...,-3.3543,40.0209,1,Daily Nation (Kenya),National,"On December 31 2019, a suspected Al Shabaab re...",1,1578512391,KEN,POINT (40.02090 -3.35430)
4,6718756,180,DRC16324,16324,31 December 2019,2019,1,Battles,Armed clash,Military Forces of Democratic Republic of Cong...,...,-1.1869,29.4461,2,Kivu Security Tracker,Local partner-Other,"On 31 December 2019, the FARDC attacked Mayi M...",0,1578943802,COD,POINT (29.44610 -1.18690)


## Base maps, Markers & Icons


In [5]:
center = [57.71, 11.98]
zoom = 12

m = Map(basemap=basemaps.Esri.WorldImagery, center=center, zoom=zoom)
m

Map(center=[57.71, 11.98], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [6]:
stamen = basemap_to_tiles(basemaps.Stamen.Toner)
m.add_layer(stamen)

In [8]:
marker = Marker(location=center, draggable=False)
m.add_layer(marker);

m

Map(center=[57.710016656706465, 11.97990417480469], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [13]:
icon_url = "http://icons.iconarchive.com/icons/pelfusion/long-shadow-media/512/Maps-Pin-Place-icon.png"

icon = Icon(icon_url=icon_url)
mark = Marker(location=[57.719503, 12.008843], icon=icon, rotation_angle=0,  rotation_origin='1280px 128px')
m.add_layer(mark);
m

Map(center=[57.69534131685637, 12.036209106445314], controls=(ZoomControl(options=['position', 'zoom_in_text',…

## Geodata and Map Styles

In [14]:
center = [4.546647, 22.373178]
zoom = 2

africa = countries[countries["continent"] == "Africa"]
africaMap = Map(basemap=basemaps.CartoDB.Positron, center=center, zoom=zoom)
geodata = GeoData(geo_dataframe = africa, 
                   style={'color': 'black', 'fillColor': '#E0D071', 'opacity':0.03, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
                   hover_style={'fillColor': '#b08a3e' , 'fillOpacity': 0.8},
                   name = 'Countries')


africaMap.add_layer(geodata)
africaMap

Map(center=[4.546647, 22.373178], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [15]:
html = HTML('''Hover Over Countries''')
html.layout.margin = '0px 20px 20px 20px'
control = WidgetControl(widget=html, position='topright')
africaMap.add_control(control)

def update_html(feature,  **kwargs):
    html.value = '''
        <h3><b>{}</b></h3>
        <h4>GDP: {:.2e} people</h4> 
        <h4>Population: {}</h4>
    '''.format(feature['properties']['name'],
               feature['properties']['gdp_md_est'],
               feature['properties']['pop_est'])

geodata.on_hover(update_html)

### Choropleth Map

In [17]:
africa = africa.set_value(60, 'name', "Ivory Coast")
pop =  dict(zip(africa['name'].tolist(), africa['pop_est'].tolist()))
africa[["name", "geometry"]].to_file("data/africa.json", driver="GeoJSON", encoding='utf-8')
geojson_data = json.load(open("data/africa.json",'r'))

C:\Users\Shakur\Anaconda3\envs\geo\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """Entry point for launching an IPython kernel.


In [18]:
for feature in geojson_data['features']:
    properties = feature['properties']
    feature.update(id=properties['name'])
    #print(feature['id'])

In [19]:
choroplethData = Choropleth(
    geo_data=geojson_data,
    choro_data=pop,
    colormap=linear.YlOrRd_04,
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'}
)

In [20]:
choro_map = Map(center = (4.546647, 22.373178), zoom = 2)
choro_map.add_layer(choroplethData)
choro_map

Map(center=[4.546647, 22.373178], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

### Clustered Maps

In [22]:
pointsMap = Map(basemap=basemaps.CartoDB.Positron, center=center, zoom=zoom)
PointData = GeoData(geo_dataframe = africa_acled.sample(500), name = 'Countries')


pointsMap.add_layer(PointData)
pointsMap

Map(center=[4.546647, 22.373178], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [40]:

clusterMap = Map(center=center, zoom=zoom)
clusterMap.add_layer(MarkerCluster(
    markers=[Marker(location=geolocation.coords[0][::-1]) for geolocation in africa_acled.sample(1000).geometry])
    )
clusterMap

Map(center=[4.546647, 22.373178], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

## Controls

In [41]:
search = SearchControl(position="topleft", 
                       url='https://nominatim.openstreetmap.org/search?format=json&q={s}', 
                       zoom=5,
                       property_name='display_name',
                       marker=marker
                      )
clusterMap.add_control(search)

clusterMap

Map(center=[4.546647, 22.373178], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', …

In [42]:
control = FullScreenControl()
clusterMap.add_control(control)